In [1]:
# MURA dataset is located in my googe drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
! pip install datasets transformers

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
from torchvision.io import read_image
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm
from timeit import default_timer as timer
from transformers import ViTFeatureExtractor, ViTImageProcessor
from transformers import ViTForImageClassification
from datasets import load_metric
from transformers import TrainingArguments
from transformers import Trainer


In [4]:
# must use cuda for ViT
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Load and Prep Data

## Import and Massage Data

In [5]:
# load train and test csv's with all image paths and targets
train_img_paths = pd.read_csv('drive/MyDrive/MURA-v1.1/train_image_paths.csv', header = None, names = ['image_path'])
test_img_paths = pd.read_csv('drive/MyDrive/MURA-v1.1/valid_image_paths.csv', header = None, names = ['image_path'])

In [6]:
len(train_img_paths)

36808

In [7]:
pd.set_option('max_colwidth', None)
train_img_paths.head()

,image_path
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png


In [8]:
# funtion to extract info from train and test dataframes
def split_img_paths(df):
    temp = df['image_path'].str.split('/', expand=True)
    new_df = pd.DataFrame(df['image_path'])
    new_df['patient_id'] = temp[3]
    new_df['study'] = temp[4].str.split('_', expand = True)[0].str[-1:].astype(int)
    new_df['region'] = temp[2].str.split('_', expand = True)[1]
    map = {'positive':'abnormal', 'negative':'normal'}
    new_df['target'] = temp[4].str.split('_', expand = True)[1]
    new_df['target'] = new_df['target'].map(map)
    #add google drive location to image path
    new_df['image_path'] = 'drive/MyDrive/' + new_df['image_path']
    
    return new_df

In [9]:
# create df with info extracted
mura_train = split_img_paths(train_img_paths)
mura_test = split_img_paths(test_img_paths)
mura_train.head()

,image_path,patient_id,study,region,target
0,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png,patient00001,1,SHOULDER,abnormal
1,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png,patient00001,1,SHOULDER,abnormal
2,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png,patient00001,1,SHOULDER,abnormal
3,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png,patient00002,1,SHOULDER,abnormal
4,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png,patient00002,1,SHOULDER,abnormal


In [10]:
# function to create df with 14 targets
def mura_to14(df):
    new_df = df.copy()
    new_df['target'] = df['target'] + '_' + df['region']
    return new_df

In [11]:
# create df with 14 targets
mura_train_14 = mura_to14(mura_train)
mura_test_14 = mura_to14(mura_test)


In [12]:
mura_train_14.head()

,image_path,patient_id,study,region,target
0,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png,patient00001,1,SHOULDER,abnormal_SHOULDER
1,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png,patient00001,1,SHOULDER,abnormal_SHOULDER
2,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png,patient00001,1,SHOULDER,abnormal_SHOULDER
3,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png,patient00002,1,SHOULDER,abnormal_SHOULDER
4,drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png,patient00002,1,SHOULDER,abnormal_SHOULDER


In [13]:
# create train df by region
wrist_train = mura_train[mura_train['region']=='WRIST']
forearm_train = mura_train[mura_train['region']=='FOREARM']
hand_train = mura_train[mura_train['region']=='HAND']
humerus_train = mura_train[mura_train['region']=='HUMERUS']
elbow_train = mura_train[mura_train['region']=='ELBOW']
shoulder_train = mura_train[mura_train['region']=='SHOULDER']
finger_train = mura_train[mura_train['region']=='FINGER']

# create test df by region
wrist_test = mura_test[mura_test['region']=='WRIST']
forearm_test = mura_test[mura_test['region']=='FOREARM']
hand_test = mura_test[mura_test['region']=='HAND']
humerus_test = mura_test[mura_test['region']=='HUMERUS']
elbow_test = mura_test[mura_test['region']=='ELBOW']
shoulder_test = mura_test[mura_test['region']=='SHOULDER']
finger_test = mura_test[mura_test['region']=='FINGER']

In [14]:
humerus_test.head()

,image_path,patient_id,study,region,target
1420,drive/MyDrive/MURA-v1.1/valid/XR_HUMERUS/patient11608/study1_positive/image1.png,patient11608,1,HUMERUS,abnormal
1421,drive/MyDrive/MURA-v1.1/valid/XR_HUMERUS/patient11608/study1_positive/image2.png,patient11608,1,HUMERUS,abnormal
1422,drive/MyDrive/MURA-v1.1/valid/XR_HUMERUS/patient11214/study1_positive/image1.png,patient11214,1,HUMERUS,abnormal
1423,drive/MyDrive/MURA-v1.1/valid/XR_HUMERUS/patient11214/study1_positive/image2.png,patient11214,1,HUMERUS,abnormal
1424,drive/MyDrive/MURA-v1.1/valid/XR_HUMERUS/patient11609/study1_positive/image1.png,patient11609,1,HUMERUS,abnormal


## Create custom Datasets and Dataloaders

https://www.learnpytorch.io/04_pytorch_custom_datasets#41-turn-loaded-images-into-dataloaders

In [15]:
# transformations for images
data_transforms = transforms.Compose([
    #
    #note that output channel is 1, will need to change and re-run for use with 3 channels
    #
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [16]:
# function to provide list of classes and a dict of classes:targets
def class_dict(two_targets):
    classes_2 = ['normal', 'abnormal']
    dict_2 = {'normal' : 0, 'abnormal': 1}
    classes_14 = ['abnormal_WRIST', 'normal_WRIST','abnormal_FOREARM', 'normal_FOREARM',
                      'abnormal_HAND', 'normal_HAND', 'abnormal_HUMERUS', 'normal_HUMERUS',
                      'abnormal_ELBOW', 'normal_ELBOW', 'abnormal_SHOULDER', 'normal_SHOULDER',
                      'abnormal_FINGER', 'normal_FINGER']
    dict_14 = {'abnormal_WRIST': 0, 'normal_WRIST': 1,'abnormal_FOREARM' : 2, 'normal_FOREARM' : 3,
                      'abnormal_HAND': 4, 'normal_HAND' : 5, 'abnormal_HUMERUS': 6, 'normal_HUMERUS' : 7,
                      'abnormal_ELBOW': 8, 'normal_ELBOW' : 9, 'abnormal_SHOULDER' : 10, 'normal_SHOULDER': 11,
                      'abnormal_FINGER': 12, 'normal_FINGER' : 13}
    if two_targets == True:
      return classes_2, dict_2
    else:
      return classes_14, dict_14

In [17]:
# create custom dataset class for our MURA dataset

class MuraDataset(Dataset):
    # change two targets to False if using 14 target data
    def __init__(self, mura_df, two_targets, transform=data_transforms):
        self.mura = mura_df
        self.transform = transform
        self.classes, self.class_to_idx = class_dict(two_targets=two_targets)

    def __len__(self):
        return len(self.mura)

    # retrives image from path in df file and target from df
    def get_image_class(self, index):
        image_path = self.mura.iloc[index]['image_path']
        class_name = self.mura.iloc[index]['target']
        return Image.open(image_path), class_name  

    def __getitem__(self, idx):
        image, class_name = self.get_image_class(idx)
        label = self.class_to_idx[class_name]

        return self.transform(image), label

In [18]:
#dict = {}
#for i in range(len(train_dataset)):
#    img, label = next(iter(train_dataloader))
#    shape = img.shape
#    dict[shape] = dict.get(shape, 0) + 1

In [19]:
# create train and test datasets for 14 target df
mura_train_14_dataset = MuraDataset(mura_df = mura_train_14, two_targets=False)
mura_test_14_dataset = MuraDataset(mura_df = mura_test_14, two_targets=False)

In [24]:
mura_train['region'].unique()

array(['SHOULDER', 'HUMERUS', 'FINGER', 'ELBOW', 'WRIST', 'FOREARM',
       'HAND'], dtype=object)

In [20]:
print(len(mura_train_14_dataset), len(mura_test_14_dataset))

36808 3197


In [21]:
# create train datasets by region
wrist_train_dataset = MuraDataset(mura_df = wrist_train, two_targets=True)
forearm_train_dataset = MuraDataset(mura_df = forearm_train, two_targets=True)
hand_train_dataset = MuraDataset(mura_df = hand_train, two_targets=True)
humerus_train_dataset = MuraDataset(mura_df = humerus_train, two_targets=True)
elbow_train_dataset = MuraDataset(mura_df = elbow_train, two_targets=True)
shoulder_train_dataset = MuraDataset(mura_df = shoulder_train, two_targets=True)
finger_train_dataset = MuraDataset(mura_df = finger_train, two_targets=True)

# create test datasets by region
wrist_test_dataset = MuraDataset(mura_df = wrist_test, two_targets=True)
forearm_test_dataset = MuraDataset(mura_df = forearm_test, two_targets=True)
hand_test_dataset = MuraDataset(mura_df = hand_test, two_targets=True)
humerus_test_dataset = MuraDataset(mura_df = humerus_test, two_targets=True)
elbow_test_dataset = MuraDataset(mura_df = elbow_test, two_targets=True)
shoulder_test_dataset = MuraDataset(mura_df = shoulder_test, two_targets=True)
finger_test_dataset = MuraDataset(mura_df = finger_test, two_targets=True)

In [ ]:
wrist_train_dataset = MuraDataset(mura_df = wrist_train, two_targets=True)
forearm_train_dataset = MuraDataset(mura_df = forearm_train, two_targets=True)
hand_train_dataset = MuraDataset(mura_df = hand_train, two_targets=True)
humerus_train_dataset = MuraDataset(mura_df = humerus_train, two_targets=True)
elbow_train_dataset = MuraDataset(mura_df = elbow_train, two_targets=True)
shoulder_train_dataset = MuraDataset(mura_df = shoulder_train, two_targets=True)
finger_train_dataset = MuraDataset(mura_df = finger_train, two_targets=True)

# create test datasets by region
wrist_test_dataset = MuraDataset(mura_df = wrist_test, two_targets=True)
forearm_test_dataset = MuraDataset(mura_df = forearm_test, two_targets=True)
hand_test_dataset = MuraDataset(mura_df = hand_test, two_targets=True)
humerus_test_dataset = MuraDataset(mura_df = humerus_test, two_targets=True)
elbow_test_dataset = MuraDataset(mura_df = elbow_test, two_targets=True)
shoulder_test_dataset = MuraDataset(mura_df = shoulder_test, two_targets=True)
finger_test_dataset = MuraDataset(mura_df = finger_test, two_targets=True)

In [42]:
wrist_len = len(wrist_train_dataset)
forearm_len = len(forearm_train_dataset)
hand_len = len(hand_train_dataset)
humerus_len = len(humerus_train_dataset)
elbow_len = len(elbow_train_dataset)
shoulder_len = len(shoulder_train_dataset)
finger_len = len(finger_train_dataset)

total = len(mura_train)

w = round(wrist_len/total,2)
fo = round(forearm_len/total,2)
ha = round(hand_len/total,2)
hu = round(humerus_len/total,2)
e = round(elbow_len/total,2)
s = round(shoulder_len/total,2)
fi = round(finger_len/total,2)

In [43]:
print(w, fo, ha, hu, e, s, fi)

0.26 0.05 0.15 0.03 0.13 0.23 0.14


In [56]:
def mura_14_sample(num_samples):
    a = wrist_train.sample(n=int(w*num_samples))
    b = forearm_train.sample(n=int(fo*num_samples)) 
    c = hand_train.sample(n=int(ha*num_samples)) 
    d = humerus_train.sample(n=int(hu*num_samples)) 
    el = elbow_train.sample(n=int(e*num_samples)) 
    f = shoulder_train.sample(n=int(s*num_samples)) 
    g = finger_train.sample(n=int(fi*num_samples)) 
    
    frames = [a,b,c,d,el,f,g]
    keys = mura_test.columns
    result = pd.concat(frames, keys=keys)

    ds = MuraDataset(mura_df = result, two_targets=False)
    return ds

In [57]:
mura_train_14_subset = mura_14_sample(500) 

In [58]:
len(mura_train_14_subset)

310

In [ ]:
#check size of datasets
print(f'wrist: {len(wrist_train_dataset)}, {len(wrist_test_dataset)}')
print(f'forearm: {len(forearm_train_dataset)}, {len(forearm_test_dataset)}')
print(f'hand: {len(hand_train_dataset)}, {len(hand_test_dataset)}')
print(f'humerus: {len(humerus_train_dataset)}, {len(humerus_test_dataset)}')
print(f'elbow: {len(elbow_train_dataset)}, {len(elbow_test_dataset)}')
print(f'shoulder: {len(shoulder_train_dataset)}, {len(shoulder_test_dataset)}')
print(f'finger: {len(finger_train_dataset)}, {len(finger_test_dataset)}')

wrist: 9752, 659
forearm: 1825, 301
hand: 5543, 460
humerus: 1272, 288
elbow: 4931, 465
shoulder: 8379, 563
finger: 5106, 461


In [ ]:
# batch size for all dataloaders we are creating
batch_size = 32

In [ ]:
# create dataloader for 14 targets dataset
mura_train_14_dataloader = DataLoader(mura_train_14_dataset, batch_size=batch_size, shuffle=True)
mura_test_14_dataloader = DataLoader(mura_test_14_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
img, label = next(iter(mura_train_14_dataloader))
img.shape, label

(torch.Size([32, 1, 224, 224]),
 tensor([ 8,  0, 10,  1,  0,  8,  1, 10,  8,  1, 13,  5, 12, 12, 10,  9,  5,  1,
          8, 12,  5,  5,  1,  1,  9,  5,  1,  6, 10,  5, 13,  9]))

In [ ]:
#create train dataloaders by region
wrist_train_dataloader = DataLoader(wrist_train_dataset, batch_size=batch_size, shuffle=True)
forearm_train_dataloader = DataLoader(forearm_train_dataset, batch_size=batch_size, shuffle=True)
hand_train_dataloader = DataLoader(hand_train_dataset, batch_size=batch_size, shuffle=True)
humerus_train_dataloader = DataLoader(humerus_train_dataset, batch_size=batch_size, shuffle=True)
elbow_train_dataloader = DataLoader(elbow_train_dataset, batch_size=batch_size, shuffle=True)
shoulder_train_dataloader = DataLoader(shoulder_train_dataset, batch_size=batch_size, shuffle=True)
finger_train_dataloader = DataLoader(finger_train_dataset, batch_size=batch_size, shuffle=True)

#create test dataloaders by region
wrist_test_dataloader = DataLoader(wrist_test_dataset, batch_size=batch_size, shuffle=True)
forearm_test_dataloader = DataLoader(forearm_test_dataset, batch_size=batch_size, shuffle=True)
hand_test_dataloader = DataLoader(hand_test_dataset, batch_size=batch_size, shuffle=True)
humerus_test_dataloader = DataLoader(humerus_test_dataset, batch_size=batch_size, shuffle=True)
elbow_test_dataloader = DataLoader(elbow_test_dataset, batch_size=batch_size, shuffle=True)
shoulder_test_dataloader = DataLoader(shoulder_test_dataset, batch_size=batch_size, shuffle=True)
finger_test_dataloader = DataLoader(finger_test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
img, label = next(iter(shoulder_train_dataloader))
img.shape, label

(torch.Size([32, 1, 224, 224]),
 tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
         1, 1, 1, 0, 1, 0, 1, 1]))

# Base Model

## Model Implementation

Implement CNN from:

https://www.learnpytorch.io/03_pytorch_computer_vision/#7-model-2-building-a-convolutional-neural-network-cnn

In [ ]:
# Create a convolutional neural network 
class base_model(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*56*56, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

In [ ]:
# define how we will measure our model accuracy
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
# define our training and test steps
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")


In [ ]:
# calculate total time training
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time/60:.3f} minutes")
    return total_time

In [ ]:
# Measure time

def run_model(train_data_loader, test_data_loader, model, loss_fn, optimizer,
              accuracy_fn=accuracy_fn, epochs=3, device=device):
    train_time_start = timer()

    # Train and test model 
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n---------")
        train_step(data_loader=train_data_loader, 
            model=model, 
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_fn,
            device=device
        )
        test_step(data_loader=test_data_loader,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device
        )

    train_time_end = timer()

    total_train_time = print_train_time(start=train_time_start,
                                              end=train_time_end,
                                              device=device)
    print(total_train_time)

## Run Models

In [ ]:
# define loss function and optimizer for all models
loss_fn_base_model = nn.CrossEntropyLoss()

lr_base_model = 0.01

# set optimizer with different model params
def set_optimizer(model, lr=lr_base_model):
    opt = torch.optim.SGD(params=model.parameters(), 
                             lr=lr)
    return opt 

epochs_base_model = 5
seed = 3

### 14 targets

In [ ]:
# create 14 target base model and move to device
torch.manual_seed(seed)

mura_14_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(mura_train_14_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(mura_14_base_model)

mura_14_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=14, bias=True)
  )
)

In [ ]:
# train and test 14 target base model
run_model(train_data_loader=mura_train_14_dataloader, test_data_loader=mura_test_14_dataloader,
          model=mura_14_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------


  0%|          | 0/5 [00:32<?, ?it/s]


KeyboardInterrupt: ignored

### Wrist

In [ ]:
# create wrist base model and move to device
torch.manual_seed(seed)

wrist_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(wrist_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(wrist_base_model)

wrist_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=2, bias=True)
  )
)

In [ ]:
# train and test wrist base model
run_model(train_data_loader=wrist_train_dataloader, test_data_loader=wrist_test_dataloader,
          model=wrist_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Forearm

In [ ]:
# create forearm base model and move to device
torch.manual_seed(seed)

forearm_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(forearm_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(forearm_base_model)

forearm_base_model

In [ ]:
# train and test forearm base model
run_model(train_data_loader=forearm_train_dataloader, test_data_loader=forearm_test_dataloader,
          model=forearm_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Hand

In [ ]:
# create hand base model and move to device
torch.manual_seed(seed)

hand_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(hand_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(hand_base_model)

hand_base_model

In [ ]:
# train and test hand base model
run_model(train_data_loader=hand_train_dataloader, test_data_loader=hand_test_dataloader,
          model=hand_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Humerus

In [ ]:
# create humerus base model and move to device
torch.manual_seed(seed)

humerus_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(humerus_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(humerus_base_model)

humerus_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test humerus base model
run_model(train_data_loader=humerus_train_dataloader, test_data_loader=humerus_test_dataloader,
          model=humerus_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Shoulder

In [ ]:
# create shoulder base model and move to device
torch.manual_seed(seed)

shoulder_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(shoulder_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(shoulder_base_model)

shoulder_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test shoulder base model
run_model(train_data_loader=shoulder_train_dataloader, test_data_loader=shoulder_test_dataloader,
          model=shoulder_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Elbow

In [ ]:
elbow_base_model.parameters

<bound method Module.parameters of base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)>

In [ ]:
# create wrist base elbow and move to device
torch.manual_seed(3)

elbow_base_model = base_model(input_shape=50176, 
    hidden_units=10, 
    output_shape=len(elbow_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(elbow_base_model)

elbow_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(50176, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test elbow base model
run_model(train_data_loader=elbow_train_dataloader, test_data_loader=elbow_test_dataloader,
          model=elbow_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

Elbow With batch size = 1
 
  0%|          | 0/3 [00:00<?, ?it/s]Epoch: 0
---------
Train loss: 2927377804200474425950208.00000 | Train accuracy: 57.96%
 33%|███▎      | 1/3 [1:32:51<3:05:42, 5571.33s/it]Test loss: 0.68236 | Test accuracy: 59.32%

Epoch: 1
---------
Train loss: 0.68688 | Train accuracy: 57.41%
 67%|██████▋   | 2/3 [1:34:09<39:00, 2340.27s/it]  Test loss: 0.70821 | Test accuracy: 59.32%

Epoch: 2
---------
Train loss: 0.68935 | Train accuracy: 57.70%
100%|██████████| 3/3 [1:35:26<00:00, 1908.91s/it]Test loss: 0.71157 | Test accuracy: 40.68%



Elbow With batch size = 32

0%|          | 0/3 [00:00<?, ?it/s]Epoch: 0
---------
Train loss: 0.67527 | Train accuracy: 59.56%
 33%|███▎      | 1/3 [19:56<39:53, 1196.83s/it]Test loss: 0.67626 | Test accuracy: 59.36%

Epoch: 1
---------
Train loss: 0.67621 | Train accuracy: 59.36%
 67%|██████▋   | 2/3 [21:01<08:50, 530.74s/it] Test loss: 0.67643 | Test accuracy: 59.17%

Epoch: 2
---------
Train loss: 0.67608 | Train accuracy: 59.36%
100%|██████████| 3/3 [22:04<00:00, 441.58s/it]Test loss: 0.67713 | Test accuracy: 58.97%


Train time on cuda: 22.079 minutes


### Finger

In [ ]:
# create finger base model and move to device
torch.manual_seed(seed)

finger_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(finger_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(finger_base_model)

finger_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test finger base model
run_model(train_data_loader=finger_train_dataloader, test_data_loader=finger_test_dataloader,
          model=finger_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

# Vision Transfomer

https://huggingface.co/blog/fine-tune-vit

 https://www.learnpytorch.io/04_pytorch_custom_datasets/#41-turn-loaded-images-into-dataloaders

In [ ]:
#pre_trained model we are using
model_name_or_path = 'google/vit-base-patch16-224-in21k'

Images are resized/rescaled to the same resolution (224x224) and normalized across the RGB channels with mean (0.5, 0.5, 0.5) and standard deviation (0.5, 0.5, 0.5).

In [ ]:
feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path)
feature_extractor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

## Create Custom Datasets

In [ ]:
# need to make images 3 channel for conform with pre trained model
ViT_transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                             transforms.ToTensor()
                                             ])

In [ ]:
f = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
#create custom dataset class to make our MURA dataset work with the pre-trained ViT
class ViT_MuraDataset(Dataset):
    def __init__(self, mura_df, two_targets, transform = ViT_transform, model_name_or_path=model_name_or_path):
        self.mura = mura_df
        self.transform = transform
        self.classes, self.class_to_idx = class_dict(two_targets=two_targets)
        #pre-process image like pre_trained model
        self.feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
        #self.feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path)

    def __len__(self):
        return len(self.mura)

    def get_image_class(self, index):
        image_path = self.mura.iloc[index]['image_path']
        class_name = self.mura.iloc[index]['target']
        #return read_image(image_path), class_name
        return Image.open(image_path), class_name  

    # helper function to preprocess image and make image label form for model
    def process_example(self, example, label):
      inputs = self.feature_extractor(example, return_tensors='pt')
      # did not want batches in pixel_values as we will collate_fn values ourselves
      inputs['pixel_values'] = torch.squeeze(inputs['pixel_values'])
      inputs['labels'] = label
      return inputs

    def __getitem__(self, idx):
        image, class_name = self.get_image_class(idx)
        image = self.transform(image)
        label = self.class_to_idx[class_name]
        return self.process_example(example=image,label=label)


In [ ]:
# create ViT test and train datasets for 14 targets
ViT_mura_train_14_dataset = ViT_MuraDataset(mura_df = mura_train_14, two_targets=False)
ViT_mura_test_14_dataset = ViT_MuraDataset(mura_df = mura_test_14, two_targets=False)

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
ViT_mura_train_14_dataset[0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [ ]:
ViT_mura_train_14_dataset[0]

{'pixel_values': tensor([[[-0.8588, -0.9216, -0.9373,  ..., -0.9451, -0.9451, -0.9373],
         [-0.9451, -0.9529, -0.9451,  ..., -0.9529, -0.9529, -0.9529],
         [-0.9451, -0.9451, -0.9451,  ..., -0.9608, -0.9529, -0.9529],
         ...,
         [-0.9059, -0.8824, -0.8588,  ..., -0.9451, -0.9451, -0.9451],
         [-0.9373, -0.9216, -0.9137,  ..., -0.9608, -0.9608, -0.9608],
         [-0.9373, -0.9373, -0.9294,  ..., -0.9686, -0.9686, -0.9686]],

        [[-0.8588, -0.9216, -0.9373,  ..., -0.9451, -0.9451, -0.9373],
         [-0.9451, -0.9529, -0.9451,  ..., -0.9529, -0.9529, -0.9529],
         [-0.9451, -0.9451, -0.9451,  ..., -0.9608, -0.9529, -0.9529],
         ...,
         [-0.9059, -0.8824, -0.8588,  ..., -0.9451, -0.9451, -0.9451],
         [-0.9373, -0.9216, -0.9137,  ..., -0.9608, -0.9608, -0.9608],
         [-0.9373, -0.9373, -0.9294,  ..., -0.9686, -0.9686, -0.9686]],

        [[-0.8588, -0.9216, -0.9373,  ..., -0.9451, -0.9451, -0.9373],
         [-0.9451, -0.9529, 

In [ ]:
# create ViT train datasets by region
ViT_shoulder_train_dataset = ViT_MuraDataset(mura_df = shoulder_train, two_targets=True)
ViT_humerus_train_dataset = ViT_MuraDataset(mura_df = humerus_train, two_targets=True)
ViT_finger_train_dataset = ViT_MuraDataset(mura_df = finger_train, two_targets=True)
ViT_elbow_train_dataset = ViT_MuraDataset(mura_df = elbow_train, two_targets=True)
ViT_wrist_train_dataset = ViT_MuraDataset(mura_df = wrist_train, two_targets=True)
ViT_forearm_train_dataset = ViT_MuraDataset(mura_df = forearm_train, two_targets=True)
ViT_hand_train_dataset = ViT_MuraDataset(mura_df = hand_train, two_targets=True)


# create ViT test datasets by region
ViT_shoulder_test_dataset = ViT_MuraDataset(mura_df = shoulder_test, two_targets=True)
ViT_humerus_test_dataset = ViT_MuraDataset(mura_df = humerus_test, two_targets=True)
ViT_finger_test_dataset = ViT_MuraDataset(mura_df = finger_test, two_targets=True)
ViT_elbow_test_dataset = ViT_MuraDataset(mura_df = elbow_test, two_targets=True)
ViT_wrist_test_dataset = ViT_MuraDataset(mura_df = wrist_test, two_targets=True)
ViT_forearm_test_dataset = ViT_MuraDataset(mura_df = forearm_test, two_targets=True)
ViT_hand_test_dataset = ViT_MuraDataset(mura_df = hand_test, two_targets=True)

In [ ]:
ViT_finger_train_dataset[0]

{'pixel_values': tensor([[[-0.9922, -0.9922, -0.9922,  ..., -1.0000, -0.9922, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -1.0000, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         ...,
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         [-0.9922, -1.0000, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],

        [[-0.9922, -0.9922, -0.9922,  ..., -1.0000, -0.9922, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -1.0000, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         ...,
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         [-0.9922, -1.0000, -0.9922,  ..., -0.9922, -0.9922, -0.9922],
         [-0.9922, -0.9922, -0.9922,  ..., -0.9922, -0.9922, -0.9922]],

        [[-0.9922, -0.9922, -0.9922,  ..., -1.0000, -0.9922, -0.9922],
         [-0.9922, -0.9922, 

In [ ]:
ViT_finger_train_dataset[0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [ ]:
# define our collate function
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
# create dataloader to test our collate_fn and dataset
ViT_mura_train_14_dataloader = DataLoader(ViT_mura_train_14_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
ViT_elbow_train_dataloader = DataLoader(ViT_elbow_train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [ ]:
ex = next(iter(ViT_mura_train_14_dataloader))
ex['pixel_values'].shape

torch.Size([32, 3, 224, 224])

In [ ]:
ex = next(iter(ViT_elbow_train_dataloader))
ex['pixel_values'].shape

torch.Size([32, 3, 224, 224])

## Run ViT Models

In [ ]:
# define metric same as base model
metric = load_metric("accuracy")
#metric = load_metric("precision")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-43-47207ee85e0a>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
# training argument for all ViTs
training_args = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="drive/MyDrive/Colab Notebooks/Training_Checkpoints",
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#labels for model
labels = class_dict(two_targets=True)[0]
labels_14 = class_dict(two_targets=False)[0]

In [ ]:
# function to run our ViT models
def run_ViT(trainer):
    train_results = trainer.train()
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    return train_results

In [ ]:
# function to run our ViT models
def resume_run_ViT(trainer):
    train_results = trainer.train(resume_from_checkpoint = True)
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    return train_results

In [ ]:
#create ViT pre-trained model for 2 targets and 14 targetsd
ViT_2_target_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

ViT_2_target_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkp

### 14 targets

In [ ]:
#create trainer for 14 target ViT model
ViT_14_target_trainer = Trainer(
    model=ViT_14_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_mura_train_14_dataset,
    eval_dataset=ViT_mura_test_14_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run 14 target ViT model
ViT_14_targets_results = run_ViT(ViT_14_target_trainer)

In [ ]:
print(ViT_14_targets_results)

### Wrist

In [ ]:
#create trainer for wrist ViT model
ViT_wrist_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_wrist_train_dataset,
    eval_dataset=ViT_wrist_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run wrist ViT model
wrist_ViT_results = run_ViT(ViT_wrist_trainer)

Step,Training Loss,Validation Loss,Accuracy
100,0.532200,0.593394,0.685888
200,0.516300,0.525224,0.729894
300,0.486700,0.499273,0.752656
400,0.403900,0.503924,0.764795
500,0.430700,0.449181,0.798179
600,0.358500,0.428768,0.817906
700,0.310800,0.506060,0.807284
800,0.384400,0.445767,0.814871
900,0.321000,0.465945,0.817906
1000,0.211700,0.510251,0.814871


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 2815208892GF
  train_loss               =       0.3589
  train_runtime            =   1:29:02.29
  train_samples_per_second =        7.302
  train_steps_per_second   =        0.228


In [ ]:
print(wrist_ViT_results)

TrainOutput(global_step=1220, training_loss=0.3588761060941415, metrics={'train_runtime': 5342.291, 'train_samples_per_second': 7.302, 'train_steps_per_second': 0.228, 'total_flos': 3.0228075308860047e+18, 'train_loss': 0.3588761060941415, 'epoch': 4.0})


### Forearm

In [ ]:
#create trainer for forearm ViT model
ViT_forearm_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_forearm_train_dataset,
    eval_dataset=ViT_forearm_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run forearm ViT model
forearm_ViT_results = run_ViT(ViT_forearm_trainer)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.305100,0.553799,0.747508
200,0.142000,0.731117,0.740864


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 526841286GF
  train_loss               =      0.3135
  train_runtime            =  0:19:20.16
  train_samples_per_second =       6.292
  train_steps_per_second   =         0.2


In [ ]:
print(forearm_ViT_results)

TrainOutput(global_step=232, training_loss=0.3134900280113878, metrics={'train_runtime': 1160.1639, 'train_samples_per_second': 6.292, 'train_steps_per_second': 0.2, 'total_flos': 5.656915241865216e+17, 'train_loss': 0.3134900280113878, 'epoch': 4.0})


### Hand

In [ ]:
#create trainer for hand ViT model
ViT_hand_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_hand_train_dataset,
    eval_dataset=ViT_hand_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run hand ViT model
hand_ViT_results = run_ViT(ViT_hand_trainer)

Step,Training Loss,Validation Loss


In [ ]:
hand_ViT_results = resume_run_ViT(ViT_hand_trainer)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/78 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1602535722GF
  train_loss               =       0.0797
  train_runtime            =   0:37:21.38
  train_samples_per_second =        9.892
  train_steps_per_second   =        0.311


In [ ]:
print(hand_ViT_results)

TrainOutput(global_step=696, training_loss=0.07973637423296084, metrics={'train_runtime': 2241.386, 'train_samples_per_second': 9.892, 'train_steps_per_second': 0.311, 'total_flos': 1.7207096293920154e+18, 'train_loss': 0.07973637423296084, 'epoch': 4.0})


### Humerus

In [ ]:
#create trainer for humerus ViT model
ViT_humerus_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_humerus_train_dataset,
    eval_dataset=ViT_humerus_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run humerus ViT model
humerus_ViT_results = run_ViT(ViT_humerus_trainer)

Step,Training Loss,Validation Loss,Accuracy
100,0.329400,0.497600,0.802083


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 369510601GF
  train_loss               =      0.4109
  train_runtime            =  0:31:41.39
  train_samples_per_second =       2.676
  train_steps_per_second   =       0.084


In [ ]:
print(humerus_ViT_results)

TrainOutput(global_step=160, training_loss=0.4108890645205975, metrics={'train_runtime': 1901.3946, 'train_samples_per_second': 2.676, 'train_steps_per_second': 0.084, 'total_flos': 3.9675898682671104e+17, 'train_loss': 0.4108890645205975, 'epoch': 4.0})



Step	Training Loss	Validation Loss	Accuracy

100	0.329400	0.497600	0.802083

***** train metrics *****

  epoch                    =         4.0

  total_flos               = 369510601GF

  train_loss               =      0.4109

  train_runtime            =  0:31:41.39

  train_samples_per_second =       2.676

  train_steps_per_second   =       0.084

### Elbow

In [ ]:
#create trainer for elbow ViT model
ViT_elbow_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_elbow_train_dataset,
    eval_dataset=ViT_elbow_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run humerus ViT model
elbow_ViT_results = run_ViT(ViT_elbow_trainer)

In [ ]:
print(elbow_ViT_results)

Elbow 


Step	Training Loss	Validation Loss	Accuracy

100	0.633700	0.669449	0.593186

200	0.662600	0.674480	0.598662

300	0.684800	0.672798	0.593186

400	0.636100	0.645050	0.611438

500	0.732300	0.673190	0.595214

600	0.676000	0.656240	0.599473

700	0.645400	0.682207	0.600284

800	0.677900	0.636125	0.639424

900	0.618500	0.612148	0.653214

1000	0.594100	0.603044	0.662949

1100	0.613800	0.587694	0.680389

1200	0.547200	0.550778	0.715271

***** train metrics *****

  epoch                    =          4.0

  total_flos               = 1423481854GF

  train_loss               =       0.6496

  train_runtime            =   2:11:14.87

  train_samples_per_second =        2.505

  train_steps_per_second   =        0.157

### Finger

In [ ]:
#create trainer for finger ViT model
ViT_finger_trainer = Trainer(
    model=ViT_2_target_model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_finger_train_dataset,
    eval_dataset=ViT_finger_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run humerus ViT model
finger_ViT_results = run_ViT(ViT_finger_trainer)

Step,Training Loss,Validation Loss,Accuracy
100,0.639200,0.609000,0.678959
200,0.490100,0.633611,0.704989
300,0.510300,0.592520,0.685466
400,0.424900,0.702305,0.670282
500,0.290000,0.855674,0.633406
600,0.294100,0.870166,0.644252


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1474000882GF
  train_loss               =       0.4408
  train_runtime            =   1:51:04.16
  train_samples_per_second =        3.065
  train_steps_per_second   =        0.096


In [ ]:
print(finger_ViT_results)

TrainOutput(global_step=640, training_loss=0.4408469617366791, metrics={'train_runtime': 6664.1651, 'train_samples_per_second': 3.065, 'train_steps_per_second': 0.096, 'total_flos': 1.582696395888427e+18, 'train_loss': 0.4408469617366791, 'epoch': 4.0})



Step	Training Loss	Validation Loss	Accuracy

100	0.639200	0.609000	0.678959

200	0.490100	0.633611	0.704989

300	0.510300	0.592520	0.685466

400	0.424900	0.702305	0.670282

500	0.290000	0.855674	0.633406

600	0.294100	0.870166	0.644252

***** train metrics *****

  epoch                    =          4.0

  total_flos               = 1474000882GF

  train_loss               =       0.4408

  train_runtime            =   1:51:04.16

  train_samples_per_second =        3.065

  train_steps_per_second   =        0.096

In [ ]:
import os

ex_img_path = 'drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png'
train_valid = 'drive/MyDrive/MURA-v1.1'
region ='drive/MyDrive/MURA-v1.1/train'
patient = 'drive/MyDrive/MURA-v1.1/train/XR_SHOULDER'
study = 'drive/MyDrive/MURA-v1.1/train/XR_SHOULDER/patient00001'

a = os.listdir(train_valid)
b = os.listdir(region)
c = os.listdir(patient)
d = os.listdir(study)


print(a,'\n\n',b,'\n\n',c,'\n\n',d)